In [1]:
spark

In [2]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [3]:
text = sc.textFile("gs://danicat/capiconf/t8.shakespeare.txt")

In [4]:
text

gs://danicat/capiconf/t8.shakespeare.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [5]:
text.take(10)

[u'This is the 100th Etext file presented by Project Gutenberg, and',
 u'is presented in cooperation with World Library, Inc., from their',
 u'Library of the Future and Shakespeare CDROMS.  Project Gutenberg',
 u'often releases Etexts that are NOT placed in the Public Domain!!',
 u'',
 u'Shakespeare',
 u'',
 u'*This Etext has certain copyright implications you should read!*',
 u'',
 u'<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM']

In [6]:
word = text.flatMap(lambda line: line.lower().split())

In [7]:
word

PythonRDD[3] at RDD at PythonRDD.scala:52

In [8]:
word.take(10)

[u'this',
 u'is',
 u'the',
 u'100th',
 u'etext',
 u'file',
 u'presented',
 u'by',
 u'project',
 u'gutenberg,']

In [9]:
word.count()

901325

In [10]:
ones = word.map(lambda w: (w, 1))

In [11]:
ones.take(10)

[(u'this', 1),
 (u'is', 1),
 (u'the', 1),
 (u'100th', 1),
 (u'etext', 1),
 (u'file', 1),
 (u'presented', 1),
 (u'by', 1),
 (u'project', 1),
 (u'gutenberg,', 1)]

In [12]:
count = ones.reduceByKey(lambda x, y: x + y)

In [13]:
count.take(10)

[(u'fawn', 11),
 (u'considered-', 1),
 (u'mustachio', 1),
 (u'protested,', 1),
 (u'sending.', 3),
 (u'offendeth', 1),
 (u'dance;', 4),
 (u'scold', 4),
 (u'nunnery', 1),
 (u'swoopstake', 1)]

In [14]:
count

PythonRDD[12] at RDD at PythonRDD.scala:52

In [16]:
print(count.toDebugString().decode('utf-8'))

(2) PythonRDD[12] at RDD at PythonRDD.scala:52 []
 |  MapPartitionsRDD[10] at mapPartitions at PythonRDD.scala:132 []
 |  ShuffledRDD[9] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[8] at reduceByKey at <ipython-input-12-8893febe9100>:1 []
    |  PythonRDD[7] at reduceByKey at <ipython-input-12-8893febe9100>:1 []
    |  gs://danicat/capiconf/t8.shakespeare.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
    |  gs://danicat/capiconf/t8.shakespeare.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []


In [17]:
count.take(10)

[(u'fawn', 11),
 (u'considered-', 1),
 (u'mustachio', 1),
 (u'protested,', 1),
 (u'sending.', 3),
 (u'offendeth', 1),
 (u'dance;', 4),
 (u'scold', 4),
 (u'nunnery', 1),
 (u'swoopstake', 1)]

In [18]:
df = count.toDF(["word", "count"])

In [19]:
df.show()

+-----------+-----+
|       word|count|
+-----------+-----+
|       fawn|   11|
|considered-|    1|
|  mustachio|    1|
| protested,|    1|
|   sending.|    3|
|  offendeth|    1|
|     dance;|    4|
|      scold|    4|
|    nunnery|    1|
| swoopstake|    1|
|  valorous,|    1|
|   whither?|    8|
|  out-night|    1|
|   benvolio|    4|
|    spider.|    1|
|    spider,|    4|
|      wood,|   13|
|      wood.|    8|
|    suck'd.|    1|
|   imagin'd|    7|
+-----------+-----+
only showing top 20 rows



In [21]:
df.describe().show()

+-------+-----------------+------------------+
|summary|             word|             count|
+-------+-----------------+------------------+
|  count|            59508|             59508|
|   mean|599.0159574468086|15.146282852725683|
| stddev|4521.544462453864| 268.6400215895862|
|    min|                "|                 1|
|    max|                }|             27549|
+-------+-----------------+------------------+



In [22]:
df.show()

+-----------+-----+
|       word|count|
+-----------+-----+
|       fawn|   11|
|considered-|    1|
|  mustachio|    1|
| protested,|    1|
|   sending.|    3|
|  offendeth|    1|
|     dance;|    4|
|      scold|    4|
|    nunnery|    1|
| swoopstake|    1|
|  valorous,|    1|
|   whither?|    8|
|  out-night|    1|
|   benvolio|    4|
|    spider.|    1|
|    spider,|    4|
|      wood,|   13|
|      wood.|    8|
|    suck'd.|    1|
|   imagin'd|    7|
+-----------+-----+
only showing top 20 rows



In [24]:
df.orderBy(df["count"].desc()).show()

+----+-----+
|word|count|
+----+-----+
| the|27549|
| and|26037|
|   i|19540|
|  to|18700|
|  of|18010|
|   a|14383|
|  my|12455|
|  in|10671|
| you|10630|
|that|10487|
|  is| 9145|
| for| 7982|
|with| 7931|
| not| 7643|
|your| 6871|
| his| 6749|
|  be| 6700|
| but| 5886|
|  he| 5884|
|this| 5882|
+----+-----+
only showing top 20 rows



In [25]:
df.createOrReplaceTempView("wordcount")

In [27]:
spark.sql("select * from wordcount order by 2 desc").show()

+----+-----+
|word|count|
+----+-----+
| the|27549|
| and|26037|
|   i|19540|
|  to|18700|
|  of|18010|
|   a|14383|
|  my|12455|
|  in|10671|
| you|10630|
|that|10487|
|  is| 9145|
| for| 7982|
|with| 7931|
| not| 7643|
|your| 6871|
| his| 6749|
|  be| 6700|
| but| 5886|
|  he| 5884|
|  as| 5882|
+----+-----+
only showing top 20 rows



In [28]:
count

PythonRDD[12] at RDD at PythonRDD.scala:52

In [29]:
df

DataFrame[word: string, count: bigint]

In [30]:
df.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



In [31]:
df.show()

+-----------+-----+
|       word|count|
+-----------+-----+
|       fawn|   11|
|considered-|    1|
|  mustachio|    1|
| protested,|    1|
|   sending.|    3|
|  offendeth|    1|
|     dance;|    4|
|      scold|    4|
|    nunnery|    1|
| swoopstake|    1|
|  valorous,|    1|
|   whither?|    8|
|  out-night|    1|
|   benvolio|    4|
|    spider.|    1|
|    spider,|    4|
|      wood,|   13|
|      wood.|    8|
|    suck'd.|    1|
|   imagin'd|    7|
+-----------+-----+
only showing top 20 rows



In [32]:
import nltk

In [33]:
from nltk.corpus import stopwords

In [36]:
sw = stopwords.words('english')

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
sw

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u"you're",
 u"you've",
 u"you'll",
 u"you'd",
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u"she's",
 u'her',
 u'hers',
 u'herself',
 u'it',
 u"it's",
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u"that'll",
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'th

In [38]:
type(sw)

list

In [39]:
df

DataFrame[word: string, count: bigint]

In [40]:
sw

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u"you're",
 u"you've",
 u"you'll",
 u"you'd",
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u"she's",
 u'her',
 u'hers',
 u'herself',
 u'it',
 u"it's",
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u"that'll",
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'th

In [43]:
swdf = spark.createDataFrame(map(lambda x: (x,), sw), ["word"])

In [44]:
swdf.show()

+----------+
|      word|
+----------+
|         i|
|        me|
|        my|
|    myself|
|        we|
|       our|
|      ours|
| ourselves|
|       you|
|    you're|
|    you've|
|    you'll|
|     you'd|
|      your|
|     yours|
|  yourself|
|yourselves|
|        he|
|       him|
|       his|
+----------+
only showing top 20 rows



In [45]:
swdf

DataFrame[word: string]

In [46]:
swdf.createOrReplaceTempView("stopwords")

In [48]:
nostop = spark.sql("select * from wordcount w where not exists (select 1 from stopwords s where s.word = w.word) order by 2 desc")

In [50]:
nostop.show(100)

+------+-----+
|  word|count|
+------+-----+
|  thou| 5138|
|   thy| 4028|
| shall| 3473|
|  good| 2560|
| would| 2240|
|   let| 2089|
| enter| 2013|
|  hath| 1902|
|  thee| 1794|
|   may| 1747|
|  i'll| 1737|
|  king| 1698|
|  upon| 1664|
|  like| 1641|
|  make| 1596|
|   one| 1486|
|  you,| 1479|
|  must| 1449|
|  know| 1402|
|  'tis| 1367|
|  come| 1359|
|   yet| 1286|
|  give| 1286|
|  love| 1279|
|  sir,| 1235|
|    go| 1224|
|   me,| 1218|
|   say| 1189|
|   see| 1169|
|  take| 1167|
|   th'| 1146|
|    us| 1075|
|   man| 1033|
|    o,| 1008|
|  tell|  993|
| first|  988|
|  well|  984|
| lord,|  977|
| never|  959|
|  time|  936|
|  doth|  912|
|  lord|  894|
|  much|  890|
| come,|  875|
|  mine|  866|
| think|  862|
|exeunt|  841|
| great|  828|
|   me.|  823|
|  you.|  813|
|  why,|  805|
|   art|  805|
| speak|  803|
|  now,|  785|
|   it.|  784|
|  him.|  755|
| scene|  753|
|  made|  749|
|  hear|  736|
|  exit|  728|
|   sir|  721|
|cannot|  719|
| sweet|  707|
| lord.|  

In [51]:
swdf

DataFrame[word: string]

In [52]:
swdf.explain()

== Physical Plan ==
Scan ExistingRDD[word#301]


In [53]:
nostop.explain()

== Physical Plan ==
*(4) Sort [count#1L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(count#1L DESC NULLS LAST, 200)
   +- SortMergeJoin [word#0], [word#301], LeftAnti
      :- *(1) Sort [word#0 ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(word#0, 200)
      :     +- Scan ExistingRDD[word#0,count#1L]
      +- *(3) Sort [word#301 ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(word#301, 200)
            +- *(2) Project [word#301]
               +- Scan ExistingRDD[word#301]


In [54]:
from pyspark.sql.functions import broadcast

In [55]:
len(sw)

179

In [56]:
bc_sw = broadcast(swdf)

In [58]:
bc_sw.createOrReplaceTempView("bc_stop")

In [59]:
bc_nostop = spark.sql("select /*+ broadcast(s) */* from wordcount w where not exists (select 1 from bc_stop s where s.word = w.word) order by 2 desc")

In [60]:
bc_nostop.show()

+-----+-----+
| word|count|
+-----+-----+
| thou| 5138|
|  thy| 4028|
|shall| 3473|
| good| 2560|
|would| 2240|
|  let| 2089|
|enter| 2013|
| hath| 1902|
| thee| 1794|
|  may| 1747|
| i'll| 1737|
| king| 1698|
| upon| 1664|
| like| 1641|
| make| 1596|
|  one| 1486|
| you,| 1479|
| must| 1449|
| know| 1402|
| 'tis| 1367|
+-----+-----+
only showing top 20 rows



In [61]:
bc_nostop.explain()

== Physical Plan ==
*(3) Sort [count#1L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(count#1L DESC NULLS LAST, 200)
   +- *(2) BroadcastHashJoin [word#0], [word#301], LeftAnti, BuildRight
      :- Scan ExistingRDD[word#0,count#1L]
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, true]))
         +- *(1) Project [word#301]
            +- Scan ExistingRDD[word#301]


In [63]:
x = bc_nostop.collect()

In [65]:
type(x)

list